### Упражнение 1

* Скачайте с портала Открытых Данных производственный календарь России: https://data.gov.ru/opendata/7708660670-proizvcalendar
* На его основе определите соответствующий календарь в Pandas: `RussianBusinessCalendar(AbstractHolidayCalendar)`
* С помощью полученного в пункте выше календаря и модуля `pd.offsets` создайте DataFrame one-hot календарных признаков:
    * День недели
    * Месяц
    * Выходной день
    * Праздничный день
    * Предпраздничный рабочий день
    * Последний день месяца
    * Последний рабочий день месяца
    * Предновогодний выходной день

In [53]:
import datetime as dt
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import re
%matplotlib inline

In [54]:
# Качаем с портала открытых данных производственный календарь России и преобразуем его
pk = pd.read_csv('data-20191112T1252-structure-20191112T1247.csv')
pk.rename(columns={'Год/Месяц': 'Year', 'Январь': '01', 'Февраль': '02', 'Март': '03', 
                    'Апрель': '04', 'Май': '05', 'Июнь': '06',
                    'Июль': '07', 'Август': '08', 'Сентябрь': '09',
                    'Октябрь': '10','Ноябрь': '11','Декабрь': '12'}, inplace=True)
pk = pk.drop(columns=['Всего рабочих дней','Всего праздничных и выходных дней', 'Количество рабочих часов при 40-часовой рабочей неделе', 
                       'Количество рабочих часов при 36-часовой рабочей неделе', 'Количество рабочих часов при 24-часовой рабочей неделе'])
#pk

In [55]:
# Функция, преобразующая датафрейм в список типа DateTime
def pk_to_dtarray(df):
  holidays = []
  st = []
  for col_name, data in df.items():
    if col_name != 'Year':
      for i in range(len(df)):
        df.loc[i,col_name] = df.loc[i, col_name].replace('+', '')
  for col_name, data in df.items():
    if col_name != 'Year':
      for i in range(len(df)):
        [holidays.append(pd.to_datetime(col_name  + '.' + x + '.' + str(df.Year[i]))) for x in df.loc[i, col_name].split(',') if ('*' not in x)] 
  return holidays

In [56]:
res = pk_to_dtarray(pk)
res.sort()

In [57]:
from pandas.tseries.holiday import AbstractHolidayCalendar, Holiday, nearest_workday, EasterMonday
from pandas.tseries.offsets import DateOffset, Day

In [58]:
class RussianBusinessCalendar(AbstractHolidayCalendar):
  start_date = dt.datetime(1999, 1, 1)
  end_date = dt.datetime(2019, 12, 31)
  rules = [Holiday('Russian Holidays', month=d.month, day=d.day, year=d.year) \
           for d in res]

russian_busday = pd.offsets.CustomBusinessDay(weekmask='Mon Tue Wed Thu Fri', 
                                              calendar=RussianBusinessCalendar())

rbc =  RussianBusinessCalendar()
russian_holiday = rbc.holidays()

In [59]:
# Create DataFrame enents
events=pd.DataFrame(index=pd.date_range(rbc.start_date, rbc.end_date))
# Определим, является ли день выходным
events.loc[events.index, 'Выходной'] = 0
events.loc[events.index.isin(russian_holiday), 'Выходной'] = 1
events['Выходной'] = events['Выходной'].astype('int')
# Это праздник?
events.loc[events.index, 'Праздник'] = 0
events.loc[(events.index.day == 1) & (events.index.month == 1), 'Праздник'] = 1
events.loc[(events.index.day == 23) & (events.index.month == 2), 'Праздник'] = 1
events.loc[(events.index.day == 8) & (events.index.month == 3), 'Праздник'] = 1
events.loc[(events.index.day == 1) & (events.index.month == 5), 'Праздник'] = 1
events.loc[(events.index.day == 9) & (events.index.month == 5), 'Праздник'] = 1
events.loc[(events.index.day == 12) & (events.index.month == 6), 'Праздник'] = 1
events.loc[(events.index.day == 4) & (events.index.month == 11), 'Праздник'] = 1
events['Праздник'] = events['Праздник'].astype('int')
# Предпраздничный рабочий день?
holidays = events[events['Праздник'] == 1].index
before_holiday = [dt.datetime(hd.year, hd.month, hd.day) + pd.Timedelta(days=1) - russian_busday \
                  for hd in holidays]
before_holiday = [d for d in before_holiday if d not in holidays]
events.loc[events.index, 'Предпраздничный рабочий день'] = 0
events.loc[events.index.isin(before_holiday), 'Предпраздничный рабочий день'] = 1
events['Предпраздничный рабочий день'] = events['Предпраздничный рабочий день'].astype('int')
# Последний день месяца?
lastdays_month = [pd.to_datetime(str(year)) + pd.offsets.MonthEnd(i) for year in events.index.year.unique() \
                  for i in range(1, 13)]
events.loc[events.index, 'Последний день месяца'] = 0
events.loc[events.index.isin(lastdays_month), 'Последний день месяца'] = 1
events['Последний день месяца'] = events['Последний день месяца'].astype('int')
# Последний рабочий день месяца
lastbusday_month = [dt.datetime(hd.year, hd.month, hd.day) + pd.Timedelta(days=1) - russian_busday \
                    for hd in lastdays_month]
events.loc[events.index, 'Последний рабочий день месяца'] = 0
events.loc[events.index.isin(lastbusday_month), 'Последний рабочий день месяца'] = 1
events['Последний рабочий день месяца'] = events['Последний рабочий день месяца'].astype('int')
# День недели
daysweek = ['Пн', 'Вт', 'Ср', 'Чт', 'Пт', 'Сб', 'Вс']
for i, name in enumerate(daysweek):
  events[name] = [1 if day == i else 0 for day in events.index.weekday]
# День месяца
daysmonth = ['Январь', 'Февраль', 'Март', 'Апрель', 'Май', 'Июнь', 'Июль', 'Август', 'Сентябрь', 'Октябрь', 'Ноябрь', 'Декабрь']
for i, name in enumerate(daysmonth):
  events[name] = [1 if month == i + 1 else 0 for month in events.index.month]
# Предновогодний выходной день
lastdays_year = [pd.to_datetime(str(year)) + pd.offsets.MonthEnd(12)  for year in events.index.year.unique()]
lastbusday_year = [dt.datetime(hd.year, hd.month, hd.day) - pd.offsets.Week(weekday=6)  for hd in lastdays_year]
events.loc[events.index, 'Предновогодний выходной день'] = 0
events.loc[events.index.isin(lastbusday_year), 'Предновогодний выходной день'] = 1
events['Предновогодний выходной день'] = events['Предновогодний выходной день'].astype('int')

In [60]:
# пример требуемого DataFrame 
events

,Выходной,Праздник,Предпраздничный рабочий день,Последний день месяца,Последний рабочий день месяца,Пн,Вт,Ср,Чт,Пт,...,Апрель,Май,Июнь,Июль,Август,Сентябрь,Октябрь,Ноябрь,Декабрь,Предновогодний выходной день
1999-01-01,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1999-01-02,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1999-01-03,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1999-01-04,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1999-01-05,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-27,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
2019-12-28,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2019-12-29,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
2019-12-30,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


### Упражнение 2

* Для временного ряда курса акций Сбербанка на основе производственного календаря России сформируйте DataFrame признаков:
    * Лаговые переменные рабочих дней с глубинами: 1, 3, 5
    * Скользящие статистики `['mean', 'median', 'max', 'min', 'std'] ` с окнами 1, 3, 5 сгрупированные по дням недели
    * Статистики `['mean', 'median', 'max', 'min', 'std'] ` в расширяющемся окне сгруппированные по рабочим, праздничным и предпраздничным дням

In [61]:
# Загружаем датасет с историческими данными курса акций Сбербанка
ts = pd.read_csv('data/sber_stocks.csv', usecols=['CLOSE', 'TRADEDATE'], 
                 index_col='TRADEDATE', parse_dates=True)
ts = ts.CLOSE
ts = pd.DataFrame(ts)
ts_res = ts.copy()
ts_res = ts_res.reset_index()

In [62]:
# Лаговые переменные рабочих дней с глубинами: 1
ts_1 = ts.shift(1, freq=pd.offsets.BDay()).reset_index()
ts_res = ts_res.merge(ts_1, left_on='CLOSE', right_on='CLOSE', suffixes=('_0', '_1'))
ts_res

,TRADEDATE_0,CLOSE,TRADEDATE_1
0,2013-03-25,98.79,2013-03-26
1,2013-03-26,97.20,2013-03-27
2,2013-03-26,97.20,2016-02-08
3,2016-02-05,97.20,2013-03-27
4,2016-02-05,97.20,2016-02-08
...,...,...,...
2171,2019-12-30,254.75,2019-12-31
2172,2020-01-06,253.90,2020-01-07
2173,2020-01-08,259.15,2020-01-09
2174,2020-01-09,257.99,2020-01-10


In [63]:
# Лаговые переменные рабочих дней с глубинами: 3
ts_3 = ts.shift(3, freq=pd.offsets.BDay()).reset_index()
ts_res = ts_res.merge(ts_3, left_on='CLOSE', right_on='CLOSE')
ts_res

,TRADEDATE_0,CLOSE,TRADEDATE_1,TRADEDATE
0,2013-03-25,98.79,2013-03-26,2013-03-28
1,2013-03-26,97.20,2013-03-27,2013-03-29
2,2013-03-26,97.20,2013-03-27,2016-02-10
3,2013-03-26,97.20,2016-02-08,2013-03-29
4,2013-03-26,97.20,2016-02-08,2016-02-10
...,...,...,...,...
3339,2019-12-30,254.75,2019-12-31,2020-01-02
3340,2020-01-06,253.90,2020-01-07,2020-01-09
3341,2020-01-08,259.15,2020-01-09,2020-01-13
3342,2020-01-09,257.99,2020-01-10,2020-01-14


In [64]:
# Лаговые переменные рабочих дней с глубинами: 5
ts_5 = ts.shift(5, freq=pd.offsets.BDay()).reset_index()
ts_res = ts_res.merge(ts_5, left_on='CLOSE', right_on='CLOSE', suffixes=('_3', '_4'))
ts_res

,TRADEDATE_0,CLOSE,TRADEDATE_1,TRADEDATE_3,TRADEDATE_4
0,2013-03-25,98.79,2013-03-26,2013-03-28,2013-04-01
1,2013-03-26,97.20,2013-03-27,2013-03-29,2013-04-02
2,2013-03-26,97.20,2013-03-27,2013-03-29,2016-02-12
3,2013-03-26,97.20,2013-03-27,2016-02-10,2013-04-02
4,2013-03-26,97.20,2013-03-27,2016-02-10,2016-02-12
...,...,...,...,...,...
6491,2019-12-30,254.75,2019-12-31,2020-01-02,2020-01-06
6492,2020-01-06,253.90,2020-01-07,2020-01-09,2020-01-13
6493,2020-01-08,259.15,2020-01-09,2020-01-13,2020-01-15
6494,2020-01-09,257.99,2020-01-10,2020-01-14,2020-01-16


In [65]:
# Скользящие статистики ['mean', 'median', 'max', 'min', 'std'] с окнами 1, 3, 5 сгрупированные по дням недели
ts['Weekday'] = ts.index.weekday
ts_rol = pd.DataFrame()
ts_rol['1mean'] = ts.groupby('Weekday')['CLOSE'].transform(lambda x: x.rolling(window=1).mean().shift(1))
ts_rol['3mean'] = ts.groupby('Weekday')['CLOSE'].transform(lambda x: x.rolling(window=3).mean().shift(1))
ts_rol['5mean']= ts.groupby('Weekday')['CLOSE'].transform(lambda x: x.rolling(window=5).mean().shift(1))
ts_rol['1median'] = ts.groupby('Weekday')['CLOSE'].transform(lambda x: x.rolling(window=1).median().shift(1))
ts_rol['3median'] = ts.groupby('Weekday')['CLOSE'].transform(lambda x: x.rolling(window=3).median().shift(1))
ts_rol['5median'] = ts.groupby('Weekday')['CLOSE'].transform(lambda x: x.rolling(window=5).median().shift(1))
ts_rol['1max'] = ts.groupby('Weekday')['CLOSE'].transform(lambda x: x.rolling(window=1).max().shift(1))
ts_rol['3max'] = ts.groupby('Weekday')['CLOSE'].transform(lambda x: x.rolling(window=3).max().shift(1))
ts_rol['5max'] = ts.groupby('Weekday')['CLOSE'].transform(lambda x: x.rolling(window=5).max().shift(1))
ts_rol['1min'] = ts.groupby('Weekday')['CLOSE'].transform(lambda x: x.rolling(window=1).min().shift(1))
ts_rol['3min'] = ts.groupby('Weekday')['CLOSE'].transform(lambda x: x.rolling(window=3).min().shift(1))
ts_rol['5min'] = ts.groupby('Weekday')['CLOSE'].transform(lambda x: x.rolling(window=5).min().shift(1))
ts_rol['1std'] = ts.groupby('Weekday')['CLOSE'].transform(lambda x: x.rolling(window=1).std().shift(1))
ts_rol['3std'] = ts.groupby('Weekday')['CLOSE'].transform(lambda x: x.rolling(window=3).std().shift(1))
ts_rol['5std'] = ts.groupby('Weekday')['CLOSE'].transform(lambda x: x.rolling(window=5).std().shift(1))
ts_rol.reset_index()
ts_res = ts_res.merge(ts_rol, left_on='TRADEDATE_0', right_on='TRADEDATE')

In [66]:
# Статистики ['mean', 'median', 'max', 'min', 'std'] в расширяющемся окне сгруппированные по рабочим, праздничным и предпраздничным дням
ts_exp = pd.DataFrame()
ts.loc[ts.index, 'Type_of_day'] = 0
# Выходной
ts.loc[ts.index.isin(russian_holiday), 'Type_of_day'] = 1
ts['Type_of_day'] = ts['Type_of_day'].astype('int')
# Праздник
ts.loc[(ts.index.day == 1) & (ts.index.month == 1), 'Type_of_day'] = 2
ts.loc[(ts.index.day == 23) & (ts.index.month == 2), 'Type_of_day'] = 2
ts.loc[(ts.index.day == 8) & (ts.index.month == 3), 'Type_of_day'] = 2
ts.loc[(ts.index.day == 1) & (ts.index.month == 5), 'Type_of_day'] = 2
ts.loc[(ts.index.day == 9) & (ts.index.month == 5), 'Type_of_day'] = 2
ts.loc[(ts.index.day == 12) & (ts.index.month == 6), 'Type_of_day'] = 2
ts.loc[(ts.index.day == 4) & (ts.index.month == 11), 'Type_of_day'] = 2
# Предпраздничный рабочий день
before_holiday = [dt.datetime(hd.year, hd.month, hd.day) + pd.Timedelta(days=1) - russian_busday for hd in holidays]
before_holiday = [d for d in before_holiday if d not in holidays]
ts.loc[ts.index.isin(before_holiday), 'Type_of_day'] = 3
ts['Type_of_day'] = ts['Type_of_day'].astype('int')
ts[ts['Type_of_day'] == 1]
ts_exp['exp_mean'] = ts.groupby('Type_of_day')['CLOSE'].expanding().mean()
ts_exp['exp_median'] = ts.groupby('Type_of_day')['CLOSE'].expanding().median()
ts_exp['exp_max'] = ts.groupby('Type_of_day')['CLOSE'].expanding().max()
ts_exp['exp_min'] = ts.groupby('Type_of_day')['CLOSE'].expanding().min()
ts_exp['exp_std'] = ts.groupby('Type_of_day')['CLOSE'].expanding().std()
ts_exp = ts_exp.reset_index()
ts_res = ts_res.merge(ts_exp, left_on='TRADEDATE_0', right_on='TRADEDATE')
ts_res.set_index('TRADEDATE_0', inplace=True)
ts_res.drop(columns=['TRADEDATE'], inplace=True)

In [67]:
ts_res

,CLOSE,TRADEDATE_1,TRADEDATE_3,TRADEDATE_4,1mean,3mean,5mean,1median,3median,5median,...,5min,1std,3std,5std,Type_of_day,exp_mean,exp_median,exp_max,exp_min,exp_std
TRADEDATE_0,,,,,,,,,,,,,,,,,,,,,
2013-03-25,98.79,2013-03-26,2013-03-28,2013-04-01,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0,98.790000,98.790,98.79,98.79,NaN
2013-03-26,97.20,2013-03-27,2013-03-29,2013-04-02,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0,97.995000,97.995,98.79,97.20,1.124300
2013-03-26,97.20,2013-03-27,2013-03-29,2016-02-12,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0,97.995000,97.995,98.79,97.20,1.124300
2013-03-26,97.20,2013-03-27,2016-02-10,2013-04-02,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0,97.995000,97.995,98.79,97.20,1.124300
2013-03-26,97.20,2013-03-27,2016-02-10,2016-02-12,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0,97.995000,97.995,98.79,97.20,1.124300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-30,254.75,2019-12-31,2020-01-02,2020-01-06,248.80,242.430000,239.808,248.80,241.45,237.98,...,233.77,NaN,5.940934,5.723047,0,145.535135,139.555,279.97,53.50,61.810856
2020-01-06,253.90,2020-01-07,2020-01-09,2020-01-13,254.75,248.333333,243.162,254.75,248.80,241.45,...,233.77,NaN,6.662269,8.583244,1,157.894167,171.295,274.60,56.37,69.114652
2020-01-08,259.15,2020-01-09,2020-01-13,2020-01-15,248.04,244.853333,240.074,248.04,246.20,240.32,...,231.59,NaN,4.032336,7.198839,1,160.630811,171.900,274.60,56.37,70.151596
